# Implementing Layer-Sequential Unit-Variance (LSUV) Weight Initialization

#### Last Time
In our [most recent notebook](https://github.com/jamesdellinger/fastai_deep_learning_course_part2_v3/blob/master) we throughly explored the world of neural network layer normalization. We paid special attention to batch normalization, first implementing it from scratch and then verifying that our home-grown attempt performed just as well as the built-in PyTorch batchnorm module.

We then experienced first-hand batchnorm's inability to successfully handle training on very small mini-batch sizes. After trying out other proposed alternatives, including [layer norm](https://arxiv.org/abs/1607.06450), [instance norm](https://arxiv.org/abs/1607.08022), and [group norm](https://arxiv.org/abs/1803.08494), we saw how with the right set of tweaks batchnorm could actually work with small batch sizes, and outperform these other approaches.

#### LSUV
Now we turn our attention to layer-sequential unit-variance initialization. Developed in 2015 [by Mishkin and Matas](https://arxiv.org/abs/1511.06422), LSUV is particularly effective for ensuring that very deep networks with complex architectures have final layer outputs that are inside or close to a standard normal distribution. 

Below we outline each step necessary to perform LSUV, implement and perform the procedure on the same MNIST dataset we've been using in each of our notebooks so far in this course. We wrap up by observing how a network whose layer weights have been initialized by LSUV performs relative to a simple baseline CNN.

#### Attribution
Virtually all the code that appears in this notebook is the creation of [Sylvain Gugger](https://www.fast.ai/about/#sylvain) and [Jeremy Howard](https://www.fast.ai/about/#jeremy). The original version of this notebook that they made for the course lecture can be found [here](https://github.com/fastai/course-v3/blob/master/nbs/dl2/07a_lsuv.ipynb). I simply re-typed, line-by-line, the pieces of logic necessary to implement the functionality that their notebook demonstrated. In some cases I changed the order of code cells and or variable names so as to fit an organization and style that seemed more intuitive to me. Any and all mistakes are my own.

On the other hand, all long-form text explanations in this notebook are solely my own creation. Writing extensive descriptions of the concepts and code in plain and simple English forces me to make sure that I actually understand how they work.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#export 
from exports.nb_07 import *

Let's begin, as always, by grabbing the [MNIST](http://yann.lecun.com/exdb/mnist/) data and constructing a simple, 5-layer CNN:

In [3]:
x_train, y_train, x_valid, y_valid = get_data()
x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh, bs = 50, 512
c = y_train.max().item() + 1
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

# Number of output channels for each convolutional layer.
n_outs = [8, 16, 32, 64, 64]  

Before we implement the tweaks necessary to enable LSUV, we'll train a simple 5-layer CNN so we can have a baseline for comparison. We won't include batchnorm layers in this first, baseline CNN.

Also, recall that our generalized ReLU class includes a hyperparameter called `sub`. This is a bias term that allows us to shift all our ReLU outputs up or down the y-axis, by a constant value. Although up until now we've used methods to define convolutional layers, this time we'll use a class so that we can have a property that stores the weights of a convolutional layer and another property that remembers the bias used to shift the ReLU activation layer outputs. Note, also, that our class includes a setter for the ReLU activation bias term. 

When we implement LSUV below it will become clear why we include all of these inside our convolutional layer class.

In [4]:
class ConvLayer(nn.Module):
    def __init__(self, n_in, n_out, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(n_in, n_out, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
        
    def forward(self, x): return self.relu(self.conv(x))
    
    @property
    def activation_bias(self): return -self.relu.sub
    @activation_bias.setter
    def activation_bias(self, v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [5]:
learn, run = get_learn_run(n_outs, data, 0.6, ConvLayer, cbs=cbfs)

In [6]:
run.fit(2, learn)

train: [2.22107921875, tensor(0.2165, device='cuda:0')]
valid: [2.006489453125, tensor(0.3546, device='cuda:0')]
train: [0.964456328125, tensor(0.6708, device='cuda:0')]
valid: [0.2753438232421875, tensor(0.8988, device='cuda:0')]


With our baseline validation accuracy score of `89%`, we're ready to implement LSUV!

Layer-Sequential Unit-Variance was introduced by Mishkin and Matas in their 2015 paper *[All You Need is a Good Init](https://arxiv.org/abs/1511.06422)* as a method for neural net layer weight initialization. The goal is to scale layer weights such that the variances of outputs from every layer are all 1.

Here's the process:
1. Initialize weights. The paper recommends orthonormal matrices. However, given that 2015 was quite a while ago in terms of "deep learning years," we will use the default init method for PyTorch `nn.Conv2d`, which is Kaiming uniform.
* Pass one batch through the model and record the outputs of all fully-connected linear and convolutional layers.
* Rescale the weights of each layer according by the standard deviation of that layer's outputs.
* Subtract the mean of each layer's activation outputs from the bias term of its ReLU activation function (using the `activation_bias` property setter we placed in the above `Conv_Layer` class) to ensure that activation outputs become normalized.
* Repeat the above two steps over and over until the mean and variance of each layer's outputs are sufficiently close to 0 and 1, respectively.
* Weights have been initialized and it's now time to train the model.

#### Step 1: Initialize network layers

In [7]:
learn, run = get_learn_run(n_outs, data, 0.6, ConvLayer, cbs=cbfs)

#### Step 2: Pass one batch through network and record linear and convolutional layer outputs

Here's a helper method that retrieves one batch from our datasets' dataloaders. The `begin_batch()` method is executed so that all preprocessing callbacks will be called:

In [8]:
#export
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

In [9]:
xb, yb = get_batch(data.train_dl, run)

Since we're only interested in the outputs of each convolutional or fully-connected linear layer, we'll create a function to recurse through our model's architecture, layer by layer, and return a list of all layers that meet this criteria.

Note that `sum(<list object>, [])` concatenates all the elements of all lists we pass to it, beginning with the initial state of the final list argument. This allows us to recurse as many times as necessary, and return a single-level, non-nested list.

In [10]:
#export
def find_modules(module, condition):
    if condition(module): return [module]
    return sum([find_modules(o, condition) for o in module.children()], [])

Our network only has five convolutional layers, and no fully-connected linear layers, so we want to add each of these five layers to our list of modules to keep track of:

In [11]:
modules = find_modules(learn.model, lambda x: isinstance(x, ConvLayer))
modules, len(modules)

([ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  ), ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  ), ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  ), ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  ), ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )], 5)

We will register hooks at each of the above five layers. This helper function will be used by the hooks to save the mean and standard deviation of the outputs at each layer: 

In [12]:
def append_stats(hook, module, inp, outp):
    d = outp.data
    hook.mean, hook.var, hook.std = d.mean().item(), d.var().item(), d.std().item()

Here are the mean and standard deviation of each layer's outputs after running one batch through the model:

In [13]:
model = learn.model.cuda()

In [14]:
with ForwardHooks(modules, append_stats) as hooks:
    model(xb)
    for hook in hooks: print(hook.mean, hook.var, hook.std)

0.3632296919822693 0.45626315474510193 0.6754725575447083
0.392016738653183 0.5587514042854309 0.7474967837333679
0.32132089138031006 0.3610764741897583 0.600896418094635
0.3329537510871887 0.30177319049835205 0.5493388772010803
0.21434631943702698 0.10931219160556793 0.33062395453453064


#### Steps 3, 4, 5: Scale each layer's weights until layer outputs are normalized

1. We first divide each layer's weights by the standard deviation of that layer's outputs.
* Then we adjust the bias term of each layer's ReLU activation function by the mean of that layer's outputs.

We repeat the above two steps over and over, until the each layer's outputs have a mean close to 0 and variance close to 1. We'll define a method just below to carry out this procedure:

In [15]:
#export
def lsuv_module(module, xb):
    h = ForwardHook(module, append_stats)
    
    while model(xb) is not None and abs(h.var - 1) > 1e-3: module.weight.data /= h.std
    while model(xb) is not None and abs(h.mean)    > 1e-3: module.activation_bias -= h.mean
    
    h.remove()
    return h.mean, h.var

Using inputs from our single batch, we execute this process on each of our model's five convolutional layers, in order:

In [16]:
for m in modules: print(lsuv_module(m, xb))

(-3.9229586690225915e-08, 0.9999997019767761)
(1.277242400732348e-08, 1.0000001192092896)
(-1.862645149230957e-09, 0.9999998807907104)
(-2.3748725652694702e-08, 1.000000238418579)
(-1.862645149230957e-08, 0.9999997019767761)


Our model's convolutional layer weights have now been initialized and it's time to see if LSUV helps us get a better performance than the validation accuracy score of `89%` that we achieved with our baseline model:

In [17]:
%time run.fit(2, learn)

train: [0.442014140625, tensor(0.8601, device='cuda:0')]
valid: [0.159346337890625, tensor(0.9510, device='cuda:0')]
train: [0.1893918359375, tensor(0.9415, device='cuda:0')]
valid: [0.08857901611328126, tensor(0.9718, device='cuda:0')]
CPU times: user 3.06 s, sys: 879 ms, total: 3.94 s
Wall time: 3.94 s


The effect of LSUV here is quite impressive: our validation accuracy score jumps almost 10% to just over `97%`! We see that LSUV helps our network converge much sooner.

In general, LSUV is especially effective for complex and deeper network architectures, where it becomes increasingly difficult to initialize layer weights and still get unit variance at the final layer's activation outputs.

### Export

In [18]:
nb_auto_export()

<IPython.core.display.Javascript object>